In [2]:
import matplotlib
from matplotlib import font_manager

!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv

fontpaths = ["C:/Users/kwon3/AppData/Local/Microsoft/Windows/Fonts/NanumGothicBold.ttf"]
font_files = font_manager.findSystemFonts(fontpaths=fontpaths)

for ff in font_files:
    font_manager.fontManager.addfont(ff)

matplotlib.rc('font', family="Malgun Gothic")
matplotlib.rcParams['axes.unicode_minus'] = False

'sudo'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.
'sudo'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

- 데이터 경로 변수

In [4]:
# DATA_PATH = "/content/drive/MyDrive/파이널프로젝트/data/"
# DATA_PATH

In [5]:
DATA_PATH = "C:/practice/finalproject/data/"
DATA_PATH

'C:/practice/finalproject/data/'

- 시드값

In [6]:
SEED = 42

- 데이터 불러오기

In [7]:
import pandas as pd
import numpy as np

train_tr = pd.read_csv(f"{DATA_PATH}store_train_transactions.csv") # 학습용 구매기록 데이터
train_target = pd.read_csv(f"{DATA_PATH}store_train.csv") # 학습용 정답 데이터
test_tr = pd.read_csv(f"{DATA_PATH}store_test_transactions.csv") # 테스트용 구매기록 데이터
submit = pd.read_csv(f"{DATA_PATH}store_submission.csv") # 제출 양식 데이터

train_tr.shape , train_target.shape , test_tr.shape , submit.shape

((523105, 7), (14940, 2), (441196, 7), (12225, 2))

- 공통 피처 파일 불러오기

In [8]:
submit = pd.read_csv(f"{DATA_PATH}store_submission.csv") # 제출 양식 데이터

In [9]:
#train_ft = pd.read_csv(f"{DATA_PATH}train_common_2.csv") # 학습 데이터(피처)
#test_ft = pd.read_csv(f"{DATA_PATH}test_common_2.csv") # 테스트 데이터(피처)

#train_ft.shape , test_ft.shape

((14940, 2331), (12225, 2331))

In [10]:
train_ft = pd.read_csv(f"{DATA_PATH}train_common_1114.csv") # 학습 데이터(피처)
test_ft = pd.read_csv(f"{DATA_PATH}test_common_1114.csv") # 테스트 데이터(피처)

train_ft.shape , test_ft.shape

((14940, 2331), (12225, 2331))

# 결측치 처리

In [11]:
mask = train_ft.isnull().sum() > 0
train_ft.isnull().sum()[mask]

Series([], dtype: int64)

In [12]:
mask = test_ft.isnull().sum() > 0
test_ft.isnull().sum()[mask]

Series([], dtype: int64)

# 특성 공학(Feature Engineering)

- ID 변수 제외

In [13]:
train_ft = train_ft.iloc[:,1:]
test_ft = test_ft.iloc[:,1:]
train_ft.shape, test_ft.shape

((14940, 2330), (12225, 2330))

- 추가 피처 만들어 보기

In [14]:
cols = [ col for col in train_ft.columns if col.startswith("pivot_지점코드_") ]

train_ft["지점코드별_구매횟수_std"] = train_ft[cols].std(axis=1)
train_ft["지점코드별_구매횟수_skew"] = train_ft[cols].skew(axis=1)
train_ft["지점코드별_구매횟수_kurt"] = train_ft[cols].kurt(axis=1)

test_ft["지점코드별_구매횟수_std"] = test_ft[cols].std(axis=1)
test_ft["지점코드별_구매횟수_skew"] = test_ft[cols].skew(axis=1)
test_ft["지점코드별_구매횟수_kurt"] = test_ft[cols].kurt(axis=1)

train_ft.shape, test_ft.shape

((14940, 2333), (12225, 2333))

In [15]:
cols = [ col for col in train_ft.columns if col.startswith("pivot_중분류_") ]  #중분류는 그대로쓰고, 다른건 차원축소하지많고,,, 

train_ft["중분류별_구매횟수_std"] = train_ft[cols].std(axis=1)
train_ft["중분류별_구매횟수_skew"] = train_ft[cols].skew(axis=1)
train_ft["중분류별_구매횟수_kurt"] = train_ft[cols].kurt(axis=1)

test_ft["중분류별_구매횟수_std"] = test_ft[cols].std(axis=1)
test_ft["중분류별_구매횟수_skew"] = test_ft[cols].skew(axis=1)
test_ft["중분류별_구매횟수_kurt"] = test_ft[cols].kurt(axis=1)

train_ft.shape, test_ft.shape

((14940, 2336), (12225, 2336))

In [16]:
cols = [ col for col in train_ft.columns if col.startswith("pivot_대분류_") ]

train_ft["대분류별_구매횟수_std"] = train_ft[cols].std(axis=1)
train_ft["대분류별_구매횟수_skew"] = train_ft[cols].skew(axis=1)
train_ft["대분류별_구매횟수_kurt"] = train_ft[cols].kurt(axis=1)

test_ft["대분류별_구매횟수_std"] = test_ft[cols].std(axis=1)
test_ft["대분류별_구매횟수_skew"] = test_ft[cols].skew(axis=1)
test_ft["대분류별_구매횟수_kurt"] = test_ft[cols].kurt(axis=1)

train_ft.shape, test_ft.shape

((14940, 2339), (12225, 2339))

In [17]:
cols = [ col for col in train_ft.columns if col.startswith("pivot_브랜드코드_") ] #써야하면 차원축소를 할것(버리던가,, 차원축소해서 옆으로붙이기) 

train_ft["브랜드코드별_구매횟수_std"] = train_ft[cols].std(axis=1)
train_ft["브랜드코드별_구매횟수_skew"] = train_ft[cols].skew(axis=1)
train_ft["브랜드코드별_구매횟수_kurt"] = train_ft[cols].kurt(axis=1)

test_ft["브랜드코드별_구매횟수_std"] = test_ft[cols].std(axis=1)
test_ft["브랜드코드별_구매횟수_skew"] = test_ft[cols].skew(axis=1)
test_ft["브랜드코드별_구매횟수_kurt"] = test_ft[cols].kurt(axis=1)

train_ft.shape, test_ft.shape

((14940, 2342), (12225, 2342))

In [18]:
cols = [ col for col in train_ft.columns if col.endswith("요일_구매비율") ]

train_ft["요일별_구매비율_std"] = train_ft[cols].std(axis=1)
train_ft["요일별_구매비율_skew"] = train_ft[cols].skew(axis=1)
train_ft["요일별_구매비율_kurt"] = train_ft[cols].kurt(axis=1)

test_ft["요일별_구매비율_std"] = test_ft[cols].std(axis=1)
test_ft["요일별_구매비율_skew"] = test_ft[cols].skew(axis=1)
test_ft["요일별_구매비율_kurt"] = test_ft[cols].kurt(axis=1)

train_ft.shape, test_ft.shape

((14940, 2345), (12225, 2345))

In [19]:
cols = [ col for col in train_ft.columns if col.endswith("월_구매비율") ]

train_ft["월별_구매비율_std"] = train_ft[cols].std(axis=1)
train_ft["월별_구매비율_skew"] = train_ft[cols].skew(axis=1)
train_ft["월별_구매비율_kurt"] = train_ft[cols].kurt(axis=1)

test_ft["월별_구매비율_std"] = test_ft[cols].std(axis=1)
test_ft["월별_구매비율_skew"] = test_ft[cols].skew(axis=1)
test_ft["월별_구매비율_kurt"] = test_ft[cols].kurt(axis=1)

train_ft.shape, test_ft.shape

((14940, 2348), (12225, 2348))

In [20]:
cols = ["봄_구매비율","여름_구매비율","가을_구매비율","겨울_구매비율"]

train_ft["계절별_구매비율_std"] = train_ft[cols].std(axis=1)
train_ft["계절별_구매비율_skew"] = train_ft[cols].skew(axis=1)
train_ft["계절별_구매비율_kurt"] = train_ft[cols].kurt(axis=1)

test_ft["계절별_구매비율_std"] = test_ft[cols].std(axis=1)
test_ft["계절별_구매비율_skew"] = test_ft[cols].skew(axis=1)
test_ft["계절별_구매비율_kurt"] = test_ft[cols].kurt(axis=1)

train_ft.shape, test_ft.shape

((14940, 2351), (12225, 2351))

In [21]:
cols = [ col for col in train_ft.columns if col.endswith("분기_구매비율") ]

train_ft["분기별_구매비율_std"] = train_ft[cols].std(axis=1)
train_ft["분기별_구매비율_skew"] = train_ft[cols].skew(axis=1)
train_ft["분기별_구매비율_kurt"] = train_ft[cols].kurt(axis=1)

test_ft["분기별_구매비율_std"] = test_ft[cols].std(axis=1)
test_ft["분기별_구매비율_skew"] = test_ft[cols].skew(axis=1)
test_ft["분기별_구매비율_kurt"] = test_ft[cols].kurt(axis=1)

train_ft.shape, test_ft.shape

((14940, 2354), (12225, 2354))

In [22]:
cols = [ col for col in train_ft.columns if col.endswith("년_구매비율") ]

train_ft["년도별_구매비율_std"] = train_ft[cols].std(axis=1)
test_ft["년도별_구매비율_std"] = test_ft[cols].std(axis=1)

train_ft.shape, test_ft.shape

((14940, 2355), (12225, 2355))

In [23]:
cols = ["봄_구매비율","여름_구매비율","가을_구매비율","겨울_구매비율"]

train_ft["계절별_구매비율_std"] = train_ft[cols].std(axis=1)
train_ft["계절별_구매비율_skew"] = train_ft[cols].skew(axis=1)
train_ft["계절별_구매비율_kurt"] = train_ft[cols].kurt(axis=1)

test_ft["계절별_구매비율_std"] = test_ft[cols].std(axis=1)
test_ft["계절별_구매비율_skew"] = test_ft[cols].skew(axis=1)
test_ft["계절별_구매비율_kurt"] = test_ft[cols].kurt(axis=1)

train_ft.shape, test_ft.shape

((14940, 2355), (12225, 2355))

In [24]:
cols = [ col for col in train_ft.columns if col.startswith("18시") ]

train_ft["18시기준_구매비율_std"] = train_ft[cols].std(axis=1)

test_ft["18시기준_구매비율_std"] = test_ft[cols].std(axis=1)

train_ft.shape, test_ft.shape

((14940, 2356), (12225, 2356))

In [25]:
# 주말에 중분류_아동용품_구매하는 비율(가족단위의 아이들용품 구매가 많지 않으까?)

train_ft["주말_중_아동용품_구매비율"] = train_ft["주말_구매비율"] * train_ft["주구매_중분류_아동용품"]

test_ft["주말_중_아동용품_구매비율"] = test_ft["주말_구매비율"] * test_ft["주구매_중분류_아동용품"]

train_ft.shape, test_ft.shape

((14940, 2357), (12225, 2357))

In [26]:
# 주말에 중분류_아동용품_구매하는 비율(가족단위의 아이들용품 구매가 많지 않으까?)

train_ft["주말_대_아동용품_구매비율"] = train_ft["주말_구매비율"] * train_ft["주구매_대분류_아동용품"]

test_ft["주말_대_아동용품_구매비율"] = test_ft["주말_구매비율"] * test_ft["주구매_대분류_아동용품"]

train_ft.shape, test_ft.shape

((14940, 2358), (12225, 2358))

In [27]:
# 18시이전이면서 주말에_중분류_아동용품_구매하는 비율
train_ft["18시이후_주말_중_아동용품_구매비율"] = train_ft["18시이후_구매비율"] * train_ft["주말_중_아동용품_구매비율"]
test_ft["18시이후_주말_중_아동용품_구매비율"] = test_ft["18시이후_구매비율"] * test_ft["주말_중_아동용품_구매비율"]

train_ft.shape, test_ft.shape

((14940, 2359), (12225, 2359))

In [28]:
train_ft.head()

총방문일수       첫구매날짜     마지막구매날짜  백화점이용기간  구매주기  주말방문일수  일별평균구매건수  구매간격_표준편차  \
0      9  2004-05-07  2005-01-16      255    28       2  1.555556  22.744789   
1     21  2004-05-11  2005-04-27      352    16       1  2.000000  11.739134   
2     55  2004-05-06  2005-04-26      356     6      11  1.854545   5.019987   
3     90  2004-05-11  2005-04-20      345     3      19  2.055556   3.137146   
4     24  2004-05-09  2005-03-17      313    13       6  1.500000  11.962731   

     평균구매요일  주구매요일  ...  계절별_구매비율_skew  계절별_구매비율_kurt  분기별_구매비율_std  \
0  3.000000      2  ...      -1.539601       2.888889      0.148690   
1  2.309524      3  ...      -0.123691      -5.290173      0.090141   
2  2.333333      0  ...       1.295669       2.179478      0.127702   
3  3.081081      4  ...       1.902132       3.642104      0.073505   
4  3.861111      4  ...       0.729678      -1.947285      0.188398   

   분기별_구매비율_skew  분기별_구매비율_kurt  년도별_구매비율_std  18시기준_구매비율_std  주말_중_아동용품_구매비율  \
0       0.000000       0.390533      0.606092        0.202031        0.030612   
1       1.658524       2.615468      0.370389        0.269374        0.000000   
2       0.238913      -4.582219      0.013865        0.499134        0.000000   
3       0.281261      -0.460929      0.080266        0.393686        0.002893   
4       1.794946       3.390359      0.471405        0.353553        0.010031   

   주말_대_아동용품_구매비율  18시이후_주말_중_아동용품_구매비율  
0        0.030612              0.010933  
1        0.000567              0.000000  
2        0.032295              0.000000  
3        0.051103              0.000641  
4        0.040123              0.002508  

[5 rows x 2359 columns]

In [29]:
mask = train_ft.isnull().sum() > 0
train_ft.isnull().sum()[mask]

Series([], dtype: int64)

In [30]:
#다이아몬드 혹은 플래티넘 등급의 아동용품 구매비율? ->의미가 없을것같긴한데,,, 




## Feature Encoding

In [31]:
train_ft["첫구매날짜"] = pd.to_datetime(train_ft["첫구매날짜"])
train_ft["마지막구매날짜"] = pd.to_datetime(train_ft["마지막구매날짜"])
test_ft["첫구매날짜"] = pd.to_datetime(test_ft["첫구매날짜"])
test_ft["마지막구매날짜"] = pd.to_datetime(test_ft["마지막구매날짜"])

train_ft["첫구매년도"] = train_ft["첫구매날짜"].dt.year
train_ft["첫구매월"] = train_ft["첫구매날짜"].dt.month
train_ft["마지막구매년도"] = train_ft["마지막구매날짜"].dt.year
train_ft["마지막구매월"] = train_ft["마지막구매날짜"].dt.month

test_ft["첫구매년도"] = test_ft["첫구매날짜"].dt.year
test_ft["첫구매월"] = test_ft["첫구매날짜"].dt.month
test_ft["마지막구매년도"] = test_ft["마지막구매날짜"].dt.year
test_ft["마지막구매월"] = test_ft["마지막구매날짜"].dt.month

train_ft.shape, test_ft.shape

((14940, 2363), (12225, 2363))

- 범주형 피처
    - 주구매지점, 주구매_중분류, 주구매_대분류, 주구매시간대, 선호브랜드코드, 고객등급

In [32]:
# %pip install category_encoders

In [33]:
import category_encoders as ce
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

oh_enc = OneHotEncoder(handle_unknown="ignore")
oh_enc_cols = ["주구매지점","주구매_대분류", "주구매시간대"]
oh_enc_data = pd.concat([train_ft[oh_enc_cols], test_ft[oh_enc_cols]])
oh_enc.fit(oh_enc_data)

train_ft[oh_enc.get_feature_names_out()] = oh_enc.transform(train_ft[oh_enc_cols]).toarray()
test_ft[oh_enc.get_feature_names_out()] = oh_enc.transform(test_ft[oh_enc_cols]).toarray()

train_ft.shape, test_ft.shape

((14940, 2404), (12225, 2404))

In [34]:
import category_encoders as ce
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

oh_enc = OneHotEncoder(handle_unknown="ignore")
oh_enc_cols = ["주구매지점", "주구매시간대"]
oh_enc_data = pd.concat([train_ft[oh_enc_cols], test_ft[oh_enc_cols]])
oh_enc.fit(oh_enc_data)

train_ft[oh_enc.get_feature_names_out()] = oh_enc.transform(train_ft[oh_enc_cols]).toarray()
test_ft[oh_enc.get_feature_names_out()] = oh_enc.transform(test_ft[oh_enc_cols]).toarray()

train_ft.shape, test_ft.shape

((14940, 2404), (12225, 2404))

In [35]:
train_ft['선호브랜드코드'] = train_ft['선호브랜드코드'].astype('category')
test_ft['선호브랜드코드'] = test_ft['선호브랜드코드'].astype('category')

combined_brands = pd.concat([train_ft['선호브랜드코드'], test_ft['선호브랜드코드']])

enc = ce.CountEncoder()
train_ft["선호브랜드코드_cnt"] = enc.fit_transform(train_ft[['선호브랜드코드']])
test_ft["선호브랜드코드_cnt"] = enc.transform(test_ft[['선호브랜드코드']])

train_ft.shape, test_ft.shape

((14940, 2405), (12225, 2405))

In [36]:
combined_categories = pd.concat([train_ft['주구매_중분류'], test_ft['주구매_중분류']])

enc = ce.count.CountEncoder()
train_ft["주구매_중분류_cnt"] = enc.fit_transform(train_ft[["주구매_중분류"]])
test_ft["주구매_중분류_cnt"] = enc.transform(test_ft[["주구매_중분류"]])

train_ft.shape, test_ft.shape

((14940, 2406), (12225, 2406))

In [37]:
# train_ft와 test_ft의 열 이름을 각각 집합으로 변환
train_columns = set(train_ft.columns)
test_columns = set(test_ft.columns)

# 공통으로 있는 열
common_columns = train_columns.intersection(test_columns)
print("공통 변수:", common_columns)

# train_ft에만 있는 열
only_in_train = train_columns - test_columns
print("train_ft에만 있는 변수:", only_in_train)

# test_ft에만 있는 열
only_in_test = test_columns - train_columns
print("test_ft에만 있는 변수:", only_in_test)

공통 변수: {'대분류별_구매횟수_skew', 'pivot_브랜드코드_5873', 'pivot_브랜드코드_5537', 'pivot_브랜드코드_5575', 'pivot_브랜드코드_5825', 'pivot_브랜드코드_5181', 'pivot_브랜드코드_6360', 'pivot_브랜드코드_6473', 'pivot_브랜드코드_6234', 'pivot_브랜드코드_6921', 'pivot_브랜드코드_5231', 'pivot_브랜드코드_6544', 'pivot_브랜드코드_5727', 'pivot_브랜드코드_5750', 'Friday_sum', 'pivot_중분류_임대슈즈', 'pivot_브랜드코드_6775', 'pivot_브랜드코드_6818', 'pivot_브랜드코드_5323', 'pivot_브랜드코드_6876', 'pivot_브랜드코드_6002', 'pivot_브랜드코드_5160', '브랜드코드_nunique', 'pivot_브랜드코드_5153', 'pivot_브랜드코드_6136', 'pivot_브랜드코드_6140', 'pivot_브랜드코드_6135', 'pivot_브랜드코드_5669', 'pivot_중분류_냉장고.세탁기', 'pivot_브랜드코드_5742', '최대환불금액', 'pivot_브랜드코드_5159', 'pivot_브랜드코드_5368', 'pivot_브랜드코드_5528', 'pivot_브랜드코드_5333', 'pivot_중분류_모피_피혁', 'pivot_브랜드코드_6132', 'pivot_중분류_NB제화', 'pivot_브랜드코드_5678', 'pivot_브랜드코드_5946', '선호브랜드코드_cnt', 'pivot_브랜드코드_5507', 'pivot_브랜드코드_5183', '요일별_구매비율_std', 'pivot_브랜드코드_5595', 'pivot_브랜드코드_5733', 'pivot_브랜드코드_5633', 'pivot_브랜드코드_5478', 'pivot_브랜드코드_5647', 'pivot_브랜드코드_5782', 'pivot_브랜드코드_5828', 'pivot

In [38]:
## 고객등급 라벨 인코딩
#from sklearn.preprocessing import LabelEncoder

#le = LabelEncoder()
#train_ft["고객등급_인코딩"] = le.fit_transform(train_ft["고객등급"])

In [39]:
#grade_mapping = {'Diamond': 4, 'Platinum': 3, 'Gold': 2, 'Black': 1, 'Noraml': 0}
#train_ft["고객등급_인코딩"] = train_ft["고객등급"].map(grade_mapping)

In [40]:
#train_ft["고객등급"]

In [41]:
#train_ft["고객등급"] = train_ft["고객등급"].str.strip() #공백제거
#grade_mapping = {'Diamond': 4, 'Platinum': 3, 'Gold': 2, 'Black': 1, 'Noraml': 0}
#train_ft["고객등급_인코딩"] = train_ft["고객등급"].replace(grade_mapping)

In [42]:
#print(train_ft["고객등급"].unique())

In [43]:
## 대소문자 차이를 없애기 위해 모두 소문자로 변환
#train_ft["고객등급"] = train_ft["고객등급"].str.lower()

# 인코딩할 딕셔너리 정의 (소문자에 맞추어 변경)
#grade_mapping = {'diamond': 4, 'platinum': 3, 'gold': 2, 'black': 1, 'normal': 0}

# "고객등급"을 딕셔너리를 통해 인코딩
#train_ft["고객등급_인코딩"] = train_ft["고객등급"].map(grade_mapping)

In [44]:
#train_ft["고객등급_인코딩"]

In [45]:
#print(train_ft["고객등급_인코딩"].unique())

In [46]:
#diamond_count = (train_ft["고객등급"] == "Diamond").sum()
#print(diamond_count)

In [47]:
#diamond_count = (train_ft["고객등급_인코딩"] == 4).sum()
#print(diamond_count)

In [48]:
#diamond_count = (train_ft["고객등급"] == "Platinum").sum()
#print(diamond_count)

In [49]:
#diamond_count = (train_ft["고객등급"] == "Gold").sum()
#print(diamond_count)

In [50]:
#diamond_count = (train_ft["고객등급"] == "Black").sum()
#print(diamond_count)

In [51]:
#diamond_count = (train_ft["고객등급"] == "Normal").sum()
#print(diamond_count)

In [52]:
## NaN 값의 갯수 확인
#nan_count = train_ft["고객등급"].isna().sum()
#print(nan_count)

In [53]:
#test_ft["고객등급"]

In [54]:
# 대소문자 차이를 없애기 위해 모두 소문자로 변환
#test_ft["고객등급"] = test_ft["고객등급"].str.lower()

# 인코딩할 딕셔너리 정의 (소문자에 맞추어 변경)
#grade_mapping = {'diamond': 4, 'platinum': 3, 'gold': 2, 'black': 1, 'normal': 0}

# "고객등급"을 딕셔너리를 통해 인코딩
#test_ft["고객등급_인코딩"] = test_ft["고객등급"].map(grade_mapping)

In [55]:
#test_ft["고객등급_인코딩"]

## 원핫인코딩

In [56]:
#import pandas as pd
#from sklearn.preprocessing import OneHotEncoder

# 예시 데이터프레임
# test_ft = pd.DataFrame({'고객등급': ['diamond', 'normal', 'gold', 'platinum', 'black']})

# OneHotEncoder 객체 생성 (drop='first'를 설정하면 첫 번째 범주를 제외한 나머지를 인코딩하여 다중 공선성 문제를 줄일 수 있음)
#encoder = OneHotEncoder(sparse=False)

# 고객등급 컬럼을 2차원 배열로 변환하여 인코딩 수행
#encoded_data = encoder.fit_transform(train_ft[["고객등급"]])

# 인코딩된 데이터를 데이터프레임으로 변환하여 기존 컬럼명에 맞춰 열 이름 지정
#encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(["고객등급"]))

# 원본 데이터프레임과 인코딩된 데이터프레임 병합
#train_ft = pd.concat([train_ft, encoded_df], axis=1).drop(columns=["고객등급"])

# 결과 출력
#print(train_ft)

In [57]:
#train_ft["고객등급"]

In [58]:
import pandas as pd

# 원핫인코딩 수행
train_ft = pd.get_dummies(train_ft, columns=['고객등급'], prefix='고객등급')

# 원핫인코딩된 열만 선택하여 True/False를 1/0으로 변환
for col in train_ft.filter(like='고객등급_').columns:
    train_ft[col] = train_ft[col].astype(int)

# 결과 확인
train_ft.head()

총방문일수      첫구매날짜    마지막구매날짜  백화점이용기간  구매주기  주말방문일수  일별평균구매건수  구매간격_표준편차  \
0      9 2004-05-07 2005-01-16      255    28       2  1.555556  22.744789   
1     21 2004-05-11 2005-04-27      352    16       1  2.000000  11.739134   
2     55 2004-05-06 2005-04-26      356     6      11  1.854545   5.019987   
3     90 2004-05-11 2005-04-20      345     3      19  2.055556   3.137146   
4     24 2004-05-09 2005-03-17      313    13       6  1.500000  11.962731   

     평균구매요일  주구매요일  ...  주구매시간대_18  주구매시간대_19  주구매시간대_20  선호브랜드코드_cnt  \
0  3.000000      2  ...        0.0        0.0        0.0            1   
1  2.309524      3  ...        1.0        0.0        0.0         7051   
2  2.333333      0  ...        0.0        0.0        0.0            8   
3  3.081081      4  ...        0.0        0.0        0.0          215   
4  3.861111      4  ...        0.0        0.0        0.0         7051   

   주구매_중분류_cnt  고객등급_Black  고객등급_Diamond  고객등급_Gold  고객등급_Normal  \
0          236           0             0          0            1   
1          521           0             0          0            1   
2           14           0             1          0            0   
3         4669           0             1          0            0   
4         2049           0             0          0            1   

   고객등급_Platinum  
0              0  
1              0  
2              0  
3              0  
4              0  

[5 rows x 2410 columns]

In [59]:
# 원핫인코딩 수행
test_ft = pd.get_dummies(test_ft, columns=['고객등급'], prefix='고객등급')

# 원핫인코딩된 열만 선택하여 True/False를 1/0으로 변환
for col in test_ft.filter(like='고객등급_').columns:
    test_ft[col] = test_ft[col].astype(int)

# 결과 확인
test_ft.head()

총방문일수      첫구매날짜    마지막구매날짜  백화점이용기간  구매주기  주말방문일수  일별평균구매건수  구매간격_표준편차  \
0      5 2004-05-16 2004-10-26      164    32       3  1.400000  30.416096   
1      3 2004-09-11 2005-02-23      166    55       1  1.333333  57.879185   
2      2 2004-08-16 2004-09-02       18     9       0  2.500000   7.361216   
3     14 2004-07-22 2005-04-19      272    19       1  1.714286  17.715031   
4      7 2004-05-25 2005-02-13      265    37       3  2.142857  36.517818   

     평균구매요일  주구매요일  ...  주구매시간대_18  주구매시간대_19  주구매시간대_20  선호브랜드코드_cnt  \
0  3.428571      1  ...        0.0        0.0        0.0       7051.0   
1  1.750000      0  ...        0.0        0.0        0.0        284.0   
2  1.200000      0  ...        0.0        0.0        0.0        577.0   
3  3.000000      3  ...        1.0        0.0        0.0          2.0   
4  4.000000      6  ...        0.0        1.0        0.0       7051.0   

   주구매_중분류_cnt  고객등급_Black  고객등급_Diamond  고객등급_Gold  고객등급_Normal  \
0       2049.0           0             0          0            1   
1         25.0           0             0          0            1   
2       4669.0           0             0          0            1   
3        521.0           0             0          0            1   
4         13.0           0             0          0            1   

   고객등급_Platinum  
0              0  
1              0  
2              0  
3              0  
4              0  

[5 rows x 2410 columns]

- 문자열 피처 삭제

In [60]:
cols = train_ft.select_dtypes(["object", "datetime"]).columns.tolist()
cols

['첫구매날짜', '마지막구매날짜', '주구매지점', '주구매_중분류', '주구매_대분류']

In [61]:
train_ft = train_ft.drop(columns=cols)
test_ft = test_ft.drop(columns=cols)
train_ft.shape, test_ft.shape

((14940, 2405), (12225, 2405))

In [62]:
train_ft.select_dtypes("object").columns , test_ft.select_dtypes("object").columns

(Index([], dtype='object'), Index([], dtype='object'))

In [63]:
train_ft.select_dtypes("datetime").columns, test_ft.select_dtypes("datetime").columns

(Index([], dtype='object'), Index([], dtype='object'))

## Feature Scaling

In [64]:
train_ft.isin([np.inf, -np.inf]).any().sum(), test_ft.isin([np.inf, -np.inf]).any().sum()

(0, 0)

In [65]:
train_ft.isna().sum().sum(), test_ft.isna().sum().sum()

(0, 0)

In [66]:
import pandas as pd

# NaN이 포함된 컬럼 찾기
nan_columns = [col for col in train_ft.columns if train_ft[col].isnull().any()]

print("NaN이 포함된 컬럼:", nan_columns)

NaN이 포함된 컬럼: []


In [67]:
#train_ft=train_ft.drop(["브랜드코드별_구매횟수_std", "브랜드코드별_구매횟수_skew", "브랜드코드별_구매횟수_kurt"], axis=1)
#test_ft=test_ft.drop(["브랜드코드별_구매횟수_std", "브랜드코드별_구매횟수_skew", "브랜드코드별_구매횟수_kurt"], axis=1)

In [68]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, PowerTransformer, Normalizer
scaler = Normalizer()

train_ft[train_ft.columns] = scaler.fit_transform(train_ft)
test_ft[test_ft.columns] = scaler.transform(test_ft)
train_ft.head()

총방문일수   백화점이용기간          구매주기        주말방문일수      일별평균구매건수     구매간격_표준편차  \
0  0.000001  0.000041  4.547622e-06  3.248301e-07  2.526457e-07  3.694097e-06   
1  0.000003  0.000059  2.663584e-06  1.664740e-07  3.329480e-07  1.954261e-06   
2  0.000002  0.000014  2.279968e-07  4.179942e-07  7.047175e-08  1.907569e-07   
3  0.000002  0.000008  6.835463e-08  4.329127e-07  4.683558e-08  7.147948e-08   
4  0.000003  0.000036  1.493358e-06  6.892421e-07  1.723105e-07  1.374203e-06   

         평균구매요일         주구매요일      월요일_구매비율      화요일_구매비율  ...     주구매시간대_18  \
0  4.872452e-07  3.248301e-07  1.160108e-08  0.000000e+00  ...  0.000000e+00   
1  3.844757e-07  4.994221e-07  7.927334e-09  4.360034e-08  ...  1.664740e-07   
2  8.866543e-08  0.000000e+00  9.686140e-09  7.078333e-09  ...  0.000000e+00   
3  7.020206e-08  9.113951e-08  1.724261e-09  2.586392e-09  ...  0.000000e+00   
4  4.435400e-07  4.594947e-07  9.572807e-09  6.381871e-09  ...  0.000000e+00   

   주구매시간대_19  주구매시간대_20   선호브랜드코드_cnt   주구매_중분류_cnt  고객등급_Black  고객등급_Diamond  \
0        0.0        0.0  1.624151e-07  3.832996e-05         0.0  0.000000e+00   
1        0.0        0.0  1.173808e-03  8.673296e-05         0.0  0.000000e+00   
2        0.0        0.0  3.039958e-07  5.319926e-07         0.0  3.799947e-08   
3        0.0        0.0  4.898749e-06  1.063826e-04         0.0  2.278488e-08   
4        0.0        0.0  8.099743e-04  2.353762e-04         0.0  0.000000e+00   

   고객등급_Gold   고객등급_Normal  고객등급_Platinum  
0        0.0  1.624151e-07            0.0  
1        0.0  1.664740e-07            0.0  
2        0.0  0.000000e+00            0.0  
3        0.0  0.000000e+00            0.0  
4        0.0  1.148737e-07            0.0  

[5 rows x 2405 columns]

In [69]:
train_ft

총방문일수   백화점이용기간          구매주기        주말방문일수      일별평균구매건수  \
0      1.461736e-06  0.000041  4.547622e-06  3.248301e-07  2.526457e-07   
1      3.495954e-06  0.000059  2.663584e-06  1.664740e-07  3.329480e-07   
2      2.089971e-06  0.000014  2.279968e-07  4.179942e-07  7.047175e-08   
3      2.050639e-06  0.000008  6.835463e-08  4.329127e-07  4.683558e-08   
4      2.756968e-06  0.000036  1.493358e-06  6.892421e-07  1.723105e-07   
...             ...       ...           ...           ...           ...   
14935  5.591171e-06  0.000152  1.677351e-05  1.242482e-06  1.173456e-06   
14936  9.588027e-07  0.000056  1.390264e-05  2.397007e-07  2.397007e-07   
14937  1.087925e-05  0.000292  2.357170e-05  1.813208e-06  1.813208e-06   
14938  2.387476e-06  0.000068  5.570778e-06  3.979127e-07  5.471300e-07   
14939  3.818270e-06  0.000097  9.545675e-06  3.818270e-07  6.872886e-07   

          구매간격_표준편차        평균구매요일         주구매요일      월요일_구매비율      화요일_구매비율  \
0      3.694097e-06  4.872452e-07  3.248301e-07  1.160108e-08  0.000000e+00   
1      1.954261e-06  3.844757e-07  4.994221e-07  7.927334e-09  4.360034e-08   
2      1.907569e-07  8.866543e-08  0.000000e+00  9.686140e-09  7.078333e-09   
3      7.147948e-08  7.020206e-08  9.113951e-08  1.724261e-09  2.586392e-09   
4      1.374203e-06  4.435400e-07  4.594947e-07  9.572807e-09  6.381871e-09   
...             ...           ...           ...           ...           ...   
14935  1.249494e-05  1.936811e-06  1.242482e-06  3.654360e-08  0.000000e+00   
14936  5.108618e-06  9.588027e-07  9.588027e-07  0.000000e+00  0.000000e+00   
14937  1.978243e-05  1.586557e-06  9.066040e-07  3.777517e-08  4.155268e-07   
14938  4.522938e-06  6.089271e-07  5.968691e-07  0.000000e+00  4.823185e-08   
14939  8.935688e-06  1.293969e-06  1.145481e-06  0.000000e+00  0.000000e+00   

       ...     주구매시간대_18     주구매시간대_19  주구매시간대_20   선호브랜드코드_cnt   주구매_중분류_cnt  \
0      ...  0.000000e+00  0.000000e+00        0.0  1.624151e-07  3.832996e-05   
1      ...  1.664740e-07  0.000000e+00        0.0  1.173808e-03  8.673296e-05   
2      ...  0.000000e+00  0.000000e+00        0.0  3.039958e-07  5.319926e-07   
3      ...  0.000000e+00  0.000000e+00        0.0  4.898749e-06  1.063826e-04   
4      ...  0.000000e+00  0.000000e+00        0.0  8.099743e-04  2.353762e-04   
...    ...           ...           ...        ...           ...           ...   
14935  ...  0.000000e+00  0.000000e+00        0.0  1.950697e-04  7.020025e-05   
14936  ...  0.000000e+00  2.397007e-07        0.0  1.869665e-05  2.397007e-07   
14937  ...  0.000000e+00  0.000000e+00        0.0  6.392465e-03  5.711605e-05   
14938  ...  0.000000e+00  0.000000e+00        0.0  2.984345e-06  1.036563e-04   
14939  ...  0.000000e+00  0.000000e+00        0.0  7.636540e-07  3.321895e-05   

       고객등급_Black  고객등급_Diamond  고객등급_Gold   고객등급_Normal  고객등급_Platinum  
0             0.0  0.000000e+00        0.0  1.624151e-07            0.0  
1             0.0  0.000000e+00        0.0  1.664740e-07            0.0  
2             0.0  3.799947e-08        0.0  0.000000e+00            0.0  
3             0.0  2.278488e-08        0.0  0.000000e+00            0.0  
4             0.0  0.000000e+00        0.0  1.148737e-07            0.0  
...           ...           ...        ...           ...            ...  
14935         0.0  0.000000e+00        0.0  6.212412e-07            0.0  
14936         0.0  0.000000e+00        0.0  2.397007e-07            0.0  
14937         0.0  0.000000e+00        0.0  9.066040e-07            0.0  
14938         0.0  0.000000e+00        0.0  1.989564e-07            0.0  
14939         0.0  0.000000e+00        0.0  3.818270e-07            0.0  

[14940 rows x 2405 columns]

In [70]:
train_ft.head()

총방문일수   백화점이용기간          구매주기        주말방문일수      일별평균구매건수     구매간격_표준편차  \
0  0.000001  0.000041  4.547622e-06  3.248301e-07  2.526457e-07  3.694097e-06   
1  0.000003  0.000059  2.663584e-06  1.664740e-07  3.329480e-07  1.954261e-06   
2  0.000002  0.000014  2.279968e-07  4.179942e-07  7.047175e-08  1.907569e-07   
3  0.000002  0.000008  6.835463e-08  4.329127e-07  4.683558e-08  7.147948e-08   
4  0.000003  0.000036  1.493358e-06  6.892421e-07  1.723105e-07  1.374203e-06   

         평균구매요일         주구매요일      월요일_구매비율      화요일_구매비율  ...     주구매시간대_18  \
0  4.872452e-07  3.248301e-07  1.160108e-08  0.000000e+00  ...  0.000000e+00   
1  3.844757e-07  4.994221e-07  7.927334e-09  4.360034e-08  ...  1.664740e-07   
2  8.866543e-08  0.000000e+00  9.686140e-09  7.078333e-09  ...  0.000000e+00   
3  7.020206e-08  9.113951e-08  1.724261e-09  2.586392e-09  ...  0.000000e+00   
4  4.435400e-07  4.594947e-07  9.572807e-09  6.381871e-09  ...  0.000000e+00   

   주구매시간대_19  주구매시간대_20   선호브랜드코드_cnt   주구매_중분류_cnt  고객등급_Black  고객등급_Diamond  \
0        0.0        0.0  1.624151e-07  3.832996e-05         0.0  0.000000e+00   
1        0.0        0.0  1.173808e-03  8.673296e-05         0.0  0.000000e+00   
2        0.0        0.0  3.039958e-07  5.319926e-07         0.0  3.799947e-08   
3        0.0        0.0  4.898749e-06  1.063826e-04         0.0  2.278488e-08   
4        0.0        0.0  8.099743e-04  2.353762e-04         0.0  0.000000e+00   

   고객등급_Gold   고객등급_Normal  고객등급_Platinum  
0        0.0  1.624151e-07            0.0  
1        0.0  1.664740e-07            0.0  
2        0.0  0.000000e+00            0.0  
3        0.0  0.000000e+00            0.0  
4        0.0  1.148737e-07            0.0  

[5 rows x 2405 columns]

# 차원축소

In [ ]:
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.model_selection import StratifiedKFold, cross_val_score
#from xgboost import XGBClassifier
#from lightgbm import LGBMClassifier

#cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

#model = LGBMClassifier(random_state=SEED)
#model.fit(train_ft_tmp_col, target)

#scores = cross_val_score(model, train_ft_tmp_col, target, cv=cv, scoring="f1_macro", n_jobs=-1)
#np.mean(scores)

- XAI 활용

In [ ]:
#%pip install shap

In [ ]:
#%pip install numpy==2.0

In [ ]:
#import shap

#explainer = shap.Explainer(model)
#shap_values = explainer(train_ft_tmp)
#shap.summary_plot(shap_values, train_ft_tmp)

- SelectFromModel

In [ ]:
#from sklearn.feature_selection import SelectFromModel

#rf = RandomForestClassifier(random_state=SEED)

## thresholds = [
##     'mean', 'median',
##     '0.5*mean', '0.8*mean', '1.2*mean', '1.5*mean', '1.8*mean', '2*mean',
##     '0.5*median', '0.8*median', '1.2*median', '1.5*median', '1.8*median', '2*median'
## ]

# thresholds = [f"{i}*mean" for i in np.arange(0.1, 1.1, 0.1)] # 0.5*mean 일 때 성능이 가장 높음
# thresholds = [f"{i}*median" for i in np.arange(1.5, 3.6, 0.1)] # 2.5*median
# thresholds = [f"{i}*median" for i in np.arange(3.5, 4.0, 0.1)] # 3.5*median
#thresholds = ['2.5*median', '3.5*median', '0.5*mean']

#for threshold in thresholds:
#    sfm = SelectFromModel(rf, threshold=threshold)
#    train_sfm = sfm.fit_transform(train_ft_tmp_col, target)

#    scores = cross_val_score(model, train_sfm, target, cv=cv, scoring="f1_macro", n_jobs=-1)
#    print(f"threshold={threshold}, 성능={np.mean(scores)}, 특성 수={train_sfm.shape[1]}")

In [ ]:
#sfm = SelectFromModel(rf, threshold='3.5*median')
#train_sfm = sfm.fit_transform(train_ft_tmp_col, target)

#scores = cross_val_score(model, train_sfm, target, cv=cv, scoring="f1_macro", n_jobs=-1)
#np.mean(scores), train_sfm.shape

- SelectKBest

In [ ]:
#from sklearn.feature_selection import SelectKBest, VarianceThreshold

#vt = VarianceThreshold(threshold=0)
#train_vt = vt.fit_transform(train_ft_tmp_col)
#test_vt = vt.transform(test_ft_tmp_col)

In [ ]:
#for k in range(300, 1001, 50):
#    skb = SelectKBest(k=k)
#    train_skb = skb.fit_transform(train_vt, target)

#    scores = cross_val_score(model, train_skb, target, cv=cv, scoring="f1_macro", n_jobs=-1)
#    print(f"특성수={k}, 정확도={np.mean(scores)}")

In [ ]:
#for k in range(590, 610):
#    skb = SelectKBest(k=k)
#    train_skb = skb.fit_transform(train_vt, target)

#    scores = cross_val_score(model, train_skb, target, cv=cv, scoring="f1_macro", n_jobs=-1)
#    print(f"특성수={k}, 정확도={np.mean(scores)}")

In [ ]:
#skb = SelectKBest(k=600)
#train_skb = skb.fit_transform(train_vt, target)

#scores = cross_val_score(model, train_skb, target, cv=cv, scoring="f1_macro", n_jobs=-1)
#np.mean(scores), train_skb.shape

- SelectPercentile

In [ ]:
#from sklearn.feature_selection import SelectPercentile

# percentiles = [i for i in range(1, 100, 5)] # 16 ~ 26 / 26 ~ 41 / 56 ~ 66 / 86 이상에서 성능점수 높게나옴
#percentiles = [i for i in range(56, 67)] # 24일 때 성능점수 가장 높음

#for percentile in percentiles:
#    sp = SelectPercentile(percentile=percentile)
#    train_sp = sp.fit_transform(train_vt, target)

#    scores = cross_val_score(model, train_sp, target, cv=cv, scoring="f1_macro", n_jobs=-1)
#    print(f"percentile={percentile}, 성능={np.mean(scores)}, 특성 수={train_sp.shape[1]}")

In [ ]:
#sp = SelectPercentile(percentile=24)
#train_sp = sp.fit_transform(train_vt, target)

#scores = cross_val_score(model, train_sp, target, cv=cv, scoring="f1_macro", n_jobs=-1)
#np.mean(scores), train_sp.shape

- SelectPercentile 에서 구한 값 근처의 피처 개수에 대해 SelectKBest

In [ ]:
#for k in range(550, 561):
#    skb = SelectKBest(k=k)
#    train_skb = skb.fit_transform(train_sp, target)

#    scores = cross_val_score(model, train_skb, target, cv=cv, scoring="f1_macro", n_jobs=-1)
#    print(f"특성수={k}, 정확도={np.mean(scores)}") # 특성수 569 일 때 최고성능점수

In [ ]:
#skb = SelectKBest(k=569)
#train_skb = skb.fit_transform(train_vt, target)

#scores = cross_val_score(model, train_skb, target, cv=cv, scoring="f1_macro", n_jobs=-1)
#np.mean(scores), train_skb.shape

- PCA

In [ ]:
#from sklearn.decomposition import PCA

#pca = PCA(train_ft.shape[1], random_state=SEED)
#pca_train = pca.fit_transform(train_ft_tmp_col)

#scores = cross_val_score(model, pca_train, target, cv=cv, scoring="f1_macro", n_jobs=-1)
#np.mean(scores), pca_train.shape

In [ ]:
#from sklearn.decomposition import PCA

## n_components를 피처 수보다 작은 값으로 설정
#pca = PCA(n_components=0.95, random_state=SEED)  # 데이터의 95% 분산 설명
#pca_train = pca.fit_transform(train_ft_tmp)

#scores = cross_val_score(model, pca_train, target, cv=cv, scoring="f1_macro", n_jobs=-1)
#np.mean(scores), pca_train.shape

In [ ]:
#cumsum = np.cumsum(pca.explained_variance_ratio_)
#n_components = np.argmax(cumsum >= 0.7) + 1
#n_components

In [ ]:
#pca = PCA(n_components, random_state=SEED)
#x_train = pca.fit_transform(train_ft_tmp)

#scores = cross_val_score(model, x_train, target, cv=cv, scoring="f1_macro", n_jobs=-1)
#np.mean(scores), x_train.shape

- LDA

In [ ]:
#from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
#from sklearn.model_selection import train_test_split

#X_train, X_val, y_train, y_val = train_test_split(
#    train_ft_tmp_col, target, test_size=0.2, random_state=SEED
#)

#lda = LDA(n_components=1)
#X_train_lda = lda.fit_transform(X_train, y_train)
#X_val_lda = lda.transform(X_val)

#scores = cross_val_score(model, X_train_lda, y_train, cv=cv, scoring="f1_macro", n_jobs=-1)
#np.mean(scores), X_train_lda.shape

In [ ]:
#scores = cross_val_score(model, X_val_lda, y_val, cv=cv, scoring='f1_macro', n_jobs=-1)
#np.mean(scores)

- RFE

In [ ]:

#from sklearn.feature_selection import RFE

#rfe = RFE(estimator=model, n_features_to_select=500, step=50)
#x_train_rfe = rfe.fit_transform(train_ft_tmp_col, target)

#scores = cross_val_score(model, x_train_rfe, target, cv=cv, scoring="f1_macro", n_jobs=-1)
#np.mean(scores), x_train_rfe.shape

- UMAP

In [ ]:
#%pip install umap-learn

In [ ]:
#import numpy as np
#from sklearn.model_selection import cross_val_score
#import umap
#from sklearn.ensemble import RandomForestClassifier  # 예시 모델
#from sklearn.datasets import make_classification

## 예시 데이터 (사용 중인 train_ft와 target을 여기에 할당)
#train_ft, target = make_classification(n_samples=1000, n_features=1000, random_state=42)

## 1. UMAP 설정 및 변환 (차원을 500으로 줄임)
#reducer = umap.UMAP(n_components=500, random_state=42)
#x_train_umap = reducer.fit_transform(train_ft_tmp_col)

## 2. 모델 정의
#model = RandomForestClassifier(random_state=42)

## 3. 교차 검증 (F1 매크로 스코어 평가)
#scores = cross_val_score(model, x_train_umap, target, cv=5, scoring="f1_macro", n_jobs=-1)
#print("Average F1 Macro Score:", np.mean(scores))
#print("Transformed Shape:", x_train_umap.shape)

- 최종적으로 모델에 넣을 학습데이터

In [ ]:
#skb = SelectKBest(k=734) # 569 or 726
#train_ft_tmptmp = skb.fit_transform(train_vt, target)
#test_ft_tmptmp = skb.transform(test_vt)
#train_ft_tmptmp.shape, test_ft_tmptmp.shape

In [ ]:
#sp = SelectPercentile(percentile=65)
#train_sp = sp.fit_transform(train_vt, target)
#test_sp = sp.transform(test_vt)
#train_sp.shape, test_sp.shape

In [ ]:
#train_ft.shape

# 0.9

In [71]:
train_ft_tmp = train_ft.drop(columns=["pivot_브랜드코드_5171",
"pivot_중분류_색조화장품",
"첫구매년도",
"선호브랜드코드",
"마지막구매년도",
"브랜드코드별_구매횟수_kurt",
"선호브랜드코드_구매비율",
"평균구매시간",
"주구매시간대",
"월별_구매비율_skew",
"월별_구매비율_std",
"성수기_구매집중도",
"전체_성수기_구매비율",
"pivot_브랜드코드_5208",
"pivot_중분류_홈데코",
"pivot_브랜드코드_6829",
"pivot_중분류_피혁",
"pivot_브랜드코드_6995",
"pivot_브랜드코드_6803",
"계절_구매변동성",
"계절별_구매비율_std",
"분기별_구매비율_std",
"브랜드코드별_구매횟수_skew",
"중분류별_구매횟수_skew",
"주말_방문_선호도",
"주말_구매비율",
"대분류별_구매횟수_skew",
"요일별_구매비율_std",
"pivot_브랜드코드_5433",
"pivot_중분류_유니섹스캐주얼",
"pivot_브랜드코드_5464",
"pivot_중분류_수입캐주얼",
"pivot_브랜드코드_6735",
"pivot_브랜드코드_6506",
"중분류별_구매횟수_kurt",
"주구매_중분류_화장품",
"요일별_구매비율_skew",
"주구매지점_이용비율",
"오후_구매비율",
"구매간격_평균_x",
"구매간격_평균_y",
"구매간격_중앙값_x",
"구매간격_중앙값_y",
"지점코드별_구매횟수_skew",
"pivot_브랜드코드_5962",
"pivot_중분류_골프(용품)",
"대분류별_구매횟수_kurt",
"pivot_브랜드코드_6377",
"pivot_브랜드코드_6053",
"구매간격_최대값_x",
"구매간격_최대값_y",
"구매주기",
"구매금액_median",
"평균구매금액",
"월별_구매비율_kurt",
"pivot_브랜드코드_6459",
"pivot_중분류_지갑_벨트",
"pivot_브랜드코드_6947",
"pivot_중분류_행사구두(5F)",
"지점코드별_구매횟수_std",
"구매건수",
"pivot_브랜드코드_5370",
"pivot_중분류_손수건",
"주구매요일",
"평균구매요일",
"pivot_브랜드코드_5930",
"pivot_중분류_TOP디자이너",
"년도별_구매비율_std",
"pivot_브랜드코드_6411",
"주구매_대분류_명품잡화",
"주구매_대분류_명품",
"대분류별_구매횟수_std",
"중분류별_구매횟수_std",
"최소환불금액",
"평균환불금액",
"요일별_구매비율_kurt",
"월별평균구매건수",
"일별평균구매건수",
"방문일수_대비_구매건수",
"구매간격_최소값_x",
"구매간격_최소값_y",
"분기별_구매비율_skew",
"계절별_구매비율_skew",
"백화점이용기간",
"pivot_브랜드코드_5969",
"pivot_중분류_유아발육기",
"18시기준_구매비율_std",
"pivot_브랜드코드_5749",
"pivot_중분류_화장품",
"총방문일수",
"최소구매금액",
"평일_18시이전_구매비율",
"18시이전_구매비율",
"pivot_브랜드코드_6977",
"pivot_브랜드코드_6528",
"연말연시_구매비율",
"12월_구매비율",
"중가제품_구매비율",
"방문지점수",
"평일_구매비율"
  ])


In [72]:
test_ft_tmp = test_ft.drop(columns=["pivot_브랜드코드_5171",
"pivot_중분류_색조화장품",
"첫구매년도",
"선호브랜드코드",
"마지막구매년도",
"브랜드코드별_구매횟수_kurt",
"선호브랜드코드_구매비율",
"평균구매시간",
"주구매시간대",
"월별_구매비율_skew",
"월별_구매비율_std",
"성수기_구매집중도",
"전체_성수기_구매비율",
"pivot_브랜드코드_5208",
"pivot_중분류_홈데코",
"pivot_브랜드코드_6829",
"pivot_중분류_피혁",
"pivot_브랜드코드_6995",
"pivot_브랜드코드_6803",
"계절_구매변동성",
"계절별_구매비율_std",
"분기별_구매비율_std",
"브랜드코드별_구매횟수_skew",
"중분류별_구매횟수_skew",
"주말_방문_선호도",
"주말_구매비율",
"대분류별_구매횟수_skew",
"요일별_구매비율_std",
"pivot_브랜드코드_5433",
"pivot_중분류_유니섹스캐주얼",
"pivot_브랜드코드_5464",
"pivot_중분류_수입캐주얼",
"pivot_브랜드코드_6735",
"pivot_브랜드코드_6506",
"중분류별_구매횟수_kurt",
"주구매_중분류_화장품",
"요일별_구매비율_skew",
"주구매지점_이용비율",
"오후_구매비율",
"구매간격_평균_x",
"구매간격_평균_y",
"구매간격_중앙값_x",
"구매간격_중앙값_y",
"지점코드별_구매횟수_skew",
"pivot_브랜드코드_5962",
"pivot_중분류_골프(용품)",
"대분류별_구매횟수_kurt",
"pivot_브랜드코드_6377",
"pivot_브랜드코드_6053",
"구매간격_최대값_x",
"구매간격_최대값_y",
"구매주기",
"구매금액_median",
"평균구매금액",
"월별_구매비율_kurt",
"pivot_브랜드코드_6459",
"pivot_중분류_지갑_벨트",
"pivot_브랜드코드_6947",
"pivot_중분류_행사구두(5F)",
"지점코드별_구매횟수_std",
"구매건수",
"pivot_브랜드코드_5370",
"pivot_중분류_손수건",
"주구매요일",
"평균구매요일",
"pivot_브랜드코드_5930",
"pivot_중분류_TOP디자이너",
"년도별_구매비율_std",
"pivot_브랜드코드_6411",
"주구매_대분류_명품잡화",
"주구매_대분류_명품",
"대분류별_구매횟수_std",
"중분류별_구매횟수_std",
"최소환불금액",
"평균환불금액",
"요일별_구매비율_kurt",
"월별평균구매건수",
"일별평균구매건수",
"방문일수_대비_구매건수",
"구매간격_최소값_x",
"구매간격_최소값_y",
"분기별_구매비율_skew",
"계절별_구매비율_skew",
"백화점이용기간",
"pivot_브랜드코드_5969",
"pivot_중분류_유아발육기",
"18시기준_구매비율_std",
"pivot_브랜드코드_5749",
"pivot_중분류_화장품",
"총방문일수",
"최소구매금액",
"평일_18시이전_구매비율",
"18시이전_구매비율",
"pivot_브랜드코드_6977",
"pivot_브랜드코드_6528",
"연말연시_구매비율",
"12월_구매비율",
"중가제품_구매비율",
"방문지점수",
"평일_구매비율"
 ])


In [ ]:
from sklearn.feature_selection import mutual_info_classif

# 상호 정보 계산
mutual_info = mutual_info_classif(train_ft, target)
print(mutual_info)

In [ ]:
# 피처 중요도 출력
feature_importance = model.feature_importances_
print(feature_importance)

In [ ]:
#교차 검증 (Cross-validation)과 피처 선택
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
score = cross_val_score(model, train_ft, target, cv=5)
print(score.mean())

# 정답 데이터

In [73]:
target = train_target["target"]
target

0        1.0
1        1.0
2        0.0
3        0.0
4        0.0
        ... 
14935    0.0
14936    0.0
14937    0.0
14938    1.0
14939    1.0
Name: target, Length: 14940, dtype: float64

# 앙상블

In [75]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import HistGradientBoostingClassifier

In [76]:

estimators = [
   # ("dt", DecisionTreeClassifier(random_state=42) ),
    #("hgb", HistGradientBoostingClassifier(max_iter=200, class_weight='balanced', random_state=42) ),
     ("hgb", HistGradientBoostingClassifier(max_iter=200, class_weight={0: 1, 1: 2}, random_state=42) ),
    #("knn", KNeighborsClassifier() ),
    ("lgb", LGBMClassifier() ),
    ("cbc", CatBoostClassifier() ),
    ("rfc", RandomForestClassifier(class_weight={0: 1 ,1: 2}, random_state=42)),
    #("lr", LogisticRegression(class_weight={0: 1, 1: 2}, random_state=42))

]



from sklearn.ensemble import VotingClassifier
parmas = {
    "estimators": estimators,
    "voting" : "soft",
    "n_jobs" : -1
}

model = VotingClassifier(**parmas)

In [77]:

from sklearn.model_selection import cross_val_score

# train_ft_tmp와 target을 numpy 배열로 변환
train_ft_X = train_ft_tmp.values  # DataFrame을 numpy 배열로 변환
target_y = target.values        # DataFrame을 numpy 배열로 변환
test_ft_X = test_ft_tmp.values

In [85]:
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, f1_score, classification_report
import pandas as pd
import numpy as np

# SMOTE 오버샘플링 적용 (훈련 데이터 전체에 적용)
smote = SMOTE(sampling_strategy=0.85, random_state=42)
train_ft_X_resampled, target_y_resampled = smote.fit_resample(train_ft_X, target_y)

# 클래스 분포 확인
print(f"Before SMOTE - target_y distribution:\n{pd.Series(target_y).value_counts()}")
print(f"After SMOTE - target_y_resampled distribution:\n{pd.Series(target_y_resampled).value_counts()}")

# 모델 생성
model = VotingClassifier(**parmas)

# 교차 검증 점수 계산 (훈련 데이터 전체로)
cv_scores = cross_val_score(model, train_ft_X_resampled, target_y_resampled, cv=5, scoring='f1_macro', n_jobs=-1)
print(f"Cross-Validation F1 Scores: {cv_scores}")
print(f"Mean CV F1 Score: {cv_scores.mean():.4f}\n")

# 모델 학습
model.fit(train_ft_X_resampled, target_y_resampled)

# 훈련 데이터 자체 평가 (검증 데이터 없이 전체 데이터 사용)
y_pred = model.predict(train_ft_X_resampled)

accuracy = accuracy_score(target_y_resampled, y_pred)
f1 = f1_score(target_y_resampled, y_pred, average='macro')

print(f"Train Accuracy: {accuracy:.4f}")
print(f"Train F1 Score: {f1:.4f}")
print("\nClassification Report:")
print(classification_report(target_y_resampled, y_pred))

# 테스트 데이터 예측
test_predictions = model.predict(test_ft_X)

Before SMOTE - target_y distribution:
0.0    9066
1.0    5874
Name: count, dtype: int64
After SMOTE - target_y_resampled distribution:
0.0    9066
1.0    7706
Name: count, dtype: int64


c:\Users\jieun\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
3 fits failed out of a total of 5.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
joblib.externals.loky.process_executor._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "c:\Users\jieun\AppData\Local\Programs\Python\Python310\lib\site-packages\joblib\_utils.py", line 72, in __call__
    return self.func(**kwargs)
  File "c:\Users\jieun\AppData\Local\Programs\Python\Python310\lib\site-packages\joblib\parallel.py", line 598, in __call__
    return [func(*args, **kwargs)
  File "c:\Users\jieun\AppData\Local\Programs\Python\Python310\lib\site-packages\

Cross-Validation F1 Scores: [0.73261151        nan        nan        nan 0.82672396]
Mean CV F1 Score: nan

Train Accuracy: 0.9456
Train F1 Score: 0.9453

Classification Report:
              precision    recall  f1-score   support

         0.0       0.96      0.94      0.95      9066
         1.0       0.93      0.95      0.94      7706

    accuracy                           0.95     16772
   macro avg       0.94      0.95      0.95     16772
weighted avg       0.95      0.95      0.95     16772



In [86]:
test_probabilities = model.predict_proba(test_ft_X)

In [87]:
test_probabilities

array([[0.87604501, 0.12395499],
       [0.54042862, 0.45957138],
       [0.41125041, 0.58874959],
       ...,
       [0.86645356, 0.13354644],
       [0.69887977, 0.30112023],
       [0.96139549, 0.03860451]])

In [91]:

# ID와 예측 확률을 결합
submission = pd.DataFrame({
    'ID': submit['ID'],  # 테스트 데이터의 ID
    'target': test_probabilities  # 클래스 1에 대한 확률값
})

In [92]:
submission.to_csv('submit_이지은.csv', index=False)

print("제출 파일이 'submission.csv'로 저장되었습니다.")

제출 파일이 'submission.csv'로 저장되었습니다.


In [ ]:
submit["target"] = test_predictions
submit
submit.to_csv(f"{DATA_PATH}submit_1114_.csv",index=False)